In [2]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
import joblib 
import os

### Code for data generation

In [ ]:
# import pandas as pd
# import numpy as np
# import random
# from datetime import datetime, timedelta

# # --- Parameters (mostly same) ---
# num_rows = 50000
# processors = ['Processor_A', 'Processor_B', 'Processor_C', 'Processor_D']

# # --- Updated: Countries and their primary currencies ---
# # Added more countries to make it more diverse and realistic
# countries = {
#     'USA': ['USD'],
#     'Germany': ['EUR'],
#     'UK': ['GBP', 'EUR'], # UK can also have EUR transactions sometimes
#     'Nigeria': ['NGN', 'USD'], # NGN primary, but USD common for international
#     'Canada': ['CAD', 'USD'],
#     'India': ['INR'],
#     'France': ['EUR'],
#     'Japan': ['JPY'],
#     'Australia': ['AUD'],
#     'Brazil': ['BRL'],
#     'Mexico': ['MXN', 'USD'],
#     'South Africa': ['ZAR'],
#     'China': ['CNY'],
#     'Egypt': ['EGP']
# }
# # Flatten the list of all possible currencies for selection if needed, or for initial definition
# all_currencies = sorted(list(set(c for sublist in countries.values() for c in sublist)))

# # Ensure our conditional adjustments reference these specific currencies if needed
# # If you have specific rules for 'USD', 'EUR', 'GBP', 'NGN' from original list, ensure they are still covered.

# statuses = ['success', 'failed']
# error_codes = ['ERR_TIMEOUT', 'ERR_DECLINED', 'ERR_NETWORK', 'ERR_UNKNOWN', 'ERR_FRAUD', 'ERR_INSUFFICIENT_FUNDS']

# payment_methods = ['Credit Card', 'Debit Card', 'Bank Transfer', 'Digital Wallet', 'Crypto']
# merchant_categories = ['Retail', 'Travel', 'Digital Goods', 'Services', 'Gaming', 'SaaS']

# # --- Base success rates for each processor ---
# base_processor_success_rates = {
#     'Processor_A': 0.95,
#     'Processor_B': 0.92,
#     'Processor_C': 0.88,
#     'Processor_D': 0.80,
# }

# # --- Base Latency Profiles (Mean, Std Dev) ---
# base_latency_profiles = {
#     'Processor_A': (100, 10),
#     'Processor_B': (110, 12),
#     'Processor_C': (105, 11),
#     'Processor_D': (120, 15),
# }

# # --- Conditional Latency Adjustments (Delta Mean Latency) ---
# # IMPORTANT: If you had currency-specific rules for countries not yet added,
# # you might need to update them or add new ones.
# conditional_latency_adjustments = {
#     # Processor A: Fast, but struggles with large international amounts and specific emerging markets
#     ('Processor_A', 'amount_bin', 'High'): 15,
#     ('Processor_A', 'currency', 'NGN'): 20,
#     ('Processor_A', 'country', 'Mexico'): 10, # Example: A bit slower for Mexico

#     # Processor B: Excels with Digital Wallets and Travel, great for EUR
#     ('Processor_B', 'payment_method', 'Digital Wallet'): -20,
#     ('Processor_B', 'merchant_category', 'Travel'): -15,
#     ('Processor_B', 'currency', 'EUR'): -10,
#     ('Processor_B', 'day_of_week_internal', 'Saturday'): -5,

#     # Processor C: Good for Digital Goods and during peak volume/hours, better for bank transfers
#     ('Processor_C', 'merchant_category', 'Digital Goods'): -12,
#     ('Processor_C', 'volume_bin', 'High'): -8,
#     ('Processor_C', 'hour_internal_bin', 'Afternoon'): -7,
#     ('Processor_C', 'payment_method', 'Bank Transfer'): -10,
#     ('Processor_C', 'currency', 'CNY'): -5, # Example: C good for China transactions

#     # Processor D: Slow generally, but surprisingly efficient for Crypto and Late Night
#     ('Processor_D', 'payment_method', 'Crypto'): -30,
#     ('Processor_D', 'hour_internal_bin', 'Late Night'): -25,
#     ('Processor_D', 'country', 'Nigeria'): -15, # Niche strength in Nigeria (lower latency)
#     ('Processor_D', 'currency', 'ZAR'): -10, # Example: D good for South African Rand

#     # General volume impact (applies to all processors unless specific rule exists)
#     ('volume_bin', 'High'): 5,
#     ('volume_bin', 'Medium'): 2,

#     # General Day of Week impact (applies to all processors unless specific rule exists)
#     ('day_of_week_internal', 'Sunday'): 3,
# }


# # --- Conditional Success Rate Adjustments ---
# # Ensure these align with your updated country/currency logic if new rules are needed.
# conditional_success_adjustments = {
#     ('Processor_B', 'currency', 'EUR'): 0.05,
#     ('Processor_C', 'volume_bin', 'High'): 0.10,
#     ('Processor_C', 'volume_bin', 'Medium'): 0.05,
#     ('Processor_A', 'amount_bin', 'High'): -0.03,
#     ('Processor_B', 'day_of_week_internal', 'Saturday'): 0.07,
#     ('Processor_B', 'day_of_week_internal', 'Sunday'): 0.07,
#     ('Processor_D', 'day_of_week_internal', 'Monday'): -0.10,
#     ('Processor_D', 'hour_internal_bin', 'Late Night'): 0.15,
#     ('Processor_A', 'payment_method', 'Credit Card'): 0.02,
#     ('Processor_B', 'payment_method', 'Digital Wallet'): 0.08,
#     ('Processor_C', 'payment_method', 'Bank Transfer'): 0.05,
#     ('Processor_D', 'payment_method', 'Debit Card'): -0.05,
#     ('Processor_A', 'merchant_category', 'Retail'): 0.03,
#     ('Processor_B', 'merchant_category', 'Travel'): 0.07,
#     ('Processor_C', 'merchant_category', 'Digital Goods'): 0.06,
#     ('Processor_D', 'merchant_category', 'Services'): -0.04,
#     ('Processor_D', 'country', 'Nigeria'): 0.05, # D has higher success in Nigeria
# }


# base_day_volume = {
#     'Monday': 65,
#     'Tuesday': 52,
#     'Wednesday': 55,
#     'Thursday': 54,
#     'Friday': 80,
#     'Saturday': 83,
#     'Sunday': 60
# }

# time_bins_internal = [0, 6, 12, 18, 24]
# time_labels_internal = ['Late Night', 'Morning', 'Afternoon', 'Evening']


# # --- Data Generation ---

# # Generate timestamps and internal time-related features
# start_date = datetime.now() - timedelta(days=30)
# timestamps = []
# internal_days_of_week = []
# internal_hours_of_day = []
# internal_hour_bins = []
# generated_countries = [] # To store the chosen country for each row
# generated_currencies = [] # To store the chosen currency for each row


# for _ in range(num_rows):
#     rand_days = random.randint(0, 29)
#     base_day = start_date + timedelta(days=rand_days)

#     hour = random.choices(
#         list(range(24)),
#         weights=[
#             0.02, 0.01, 0.01, 0.01, 0.01, 0.02,
#             0.05, 0.10, 0.15, 0.10, 0.10, 0.10,
#             0.12, 0.12, 0.10, 0.08, 0.05, 0.03,
#             0.02, 0.02, 0.01, 0.01, 0.01, 0.01
#         ]
#     )[0]
#     minute = random.randint(0, 59)
#     second = random.randint(0, 59)

#     dt = base_day.replace(hour=hour, minute=minute, second=second)
#     timestamps.append(dt)
#     internal_days_of_week.append(dt.strftime("%A"))
#     internal_hours_of_day.append(hour)
    
#     current_time_label = pd.cut([hour], bins=time_bins_internal, labels=time_labels_internal, right=False, include_lowest=True)[0]
#     internal_hour_bins.append(current_time_label)

#     # --- NEW: Select country and then a corresponding currency ---
#     country_choice = random.choice(list(countries.keys()))
#     generated_countries.append(country_choice)
    
#     # Select a currency from the list associated with the chosen country
#     currency_choice = random.choice(countries[country_choice])
#     generated_currencies.append(currency_choice)


# # Generate processor choices (equal distribution for better conditional learning)
# processor_column = np.random.choice(processors, size=num_rows, p=[0.25, 0.25, 0.25, 0.25])

# # Create base data dictionary
# data = {
#     'transaction_id': [f"txn_{i:06d}" for i in range(num_rows)],
#     'amount': np.round(np.random.uniform(5, 1000, num_rows), 2),
#     'currency': generated_currencies, # Use the generated currencies
#     'country': generated_countries,   # Use the generated countries
#     'processor': processor_column,
#     'timestamp': timestamps,
#     'payment_method': np.random.choice(payment_methods, num_rows),
#     'merchant_category': np.random.choice(merchant_categories, num_rows),
# }

# amount_bins = [0, 100, 500, 1000, 5000, 10000]
# amount_labels = ['Low', 'Medium', 'High', 'Very High', 'Extreme']
# temp_amount_bin = pd.cut(data['amount'], bins=amount_bins, labels=amount_labels, right=True, include_lowest=True)


# volume_list = []
# for i in range(num_rows):
#     base_volume = base_day_volume[internal_days_of_week[i]] 
#     hour_adj = 0
#     if 8 <= internal_hours_of_day[i] < 18:
#         hour_adj = 15
#     elif 18 <= internal_hours_of_day[i] < 22:
#         hour_adj = 5
    
#     volume = int(np.clip(np.random.normal(loc=base_volume + hour_adj, scale=15), 1, 200))
#     volume_list.append(volume)
# data['volume'] = volume_list

# volume_bins = [0, 50, 100, 150, 200]
# volume_labels = ['Low', 'Medium', 'High', 'Very High']
# temp_volume_bin = pd.cut(data['volume'], bins=volume_bins, labels=volume_labels, right=True, include_lowest=True)


# # Generate status and latency
# status_list = []
# error_code_list = []
# latency_list = []

# for i in range(num_rows):
#     current_processor = data['processor'][i]
#     current_currency = data['currency'][i]
#     current_country = data['country'][i]
#     current_payment_method = data['payment_method'][i]
#     current_merchant_category = data['merchant_category'][i]
    
#     current_day_of_week_internal = internal_days_of_week[i] 
#     current_hour_bin_internal = internal_hour_bins[i]
#     current_amount_bin = temp_amount_bin[i] 
#     current_volume_bin = temp_volume_bin[i] 

#     # --- Calculate Latency ---
#     base_mean_latency, std_dev_latency = base_latency_profiles[current_processor]
#     adjusted_mean_latency = base_mean_latency
    
#     for cond_key, adjustment_value in conditional_latency_adjustments.items():
#         if len(cond_key) == 3:
#             processor_key, feature_name, feature_value = cond_key
#             if processor_key == current_processor:
#                 if feature_name == 'amount_bin' and feature_value == current_amount_bin:
#                     adjusted_mean_latency += adjustment_value
#                 elif feature_name == 'currency' and feature_value == current_currency:
#                     adjusted_mean_latency += adjustment_value
#                 elif feature_name == 'payment_method' and feature_value == current_payment_method:
#                     adjusted_mean_latency += adjustment_value
#                 elif feature_name == 'merchant_category' and feature_value == current_merchant_category:
#                     adjusted_mean_latency += adjustment_value
#                 elif feature_name == 'day_of_week_internal' and feature_value == current_day_of_week_internal:
#                     adjusted_mean_latency += adjustment_value
#                 elif feature_name == 'hour_internal_bin' and feature_value == current_hour_bin_internal:
#                     adjusted_mean_latency += adjustment_value
#                 elif feature_name == 'country' and feature_value == current_country:
#                     adjusted_mean_latency += adjustment_value
#         elif len(cond_key) == 2:
#             feature_name, feature_value = cond_key
#             if feature_name == 'volume_bin' and feature_value == current_volume_bin:
#                 adjusted_mean_latency += adjustment_value
#             elif feature_name == 'day_of_week_internal' and feature_value == current_day_of_week_internal:
#                 adjusted_mean_latency += adjustment_value

#     adjusted_mean_latency = max(50, adjusted_mean_latency)

#     latency = max(0, np.random.normal(loc=adjusted_mean_latency, scale=std_dev_latency))
    
#     # --- Calculate Status ---
#     success_prob = base_processor_success_rates[current_processor]

#     for cond_key, adjustment_value in conditional_success_adjustments.items():
#         if len(cond_key) == 3:
#             processor_key, feature_name, feature_value = cond_key
#             if processor_key == current_processor:
#                 if feature_name == 'currency' and feature_value == current_currency:
#                     success_prob += adjustment_value
#                 elif feature_name == 'day_of_week_internal' and feature_value == current_day_of_week_internal:
#                     success_prob += adjustment_value
#                 elif feature_name == 'hour_internal_bin' and feature_value == current_hour_bin_internal:
#                     success_prob += adjustment_value
#                 elif feature_name == 'amount_bin' and feature_value == current_amount_bin:
#                     success_prob += adjustment_value
#                 elif feature_name == 'volume_bin' and feature_value == current_volume_bin:
#                     success_prob += adjustment_value
#                 elif feature_name == 'payment_method' and feature_value == current_payment_method:
#                     success_prob += adjustment_value
#                 elif feature_name == 'merchant_category' and feature_value == current_merchant_category:
#                     success_prob += adjustment_value
#                 elif feature_name == 'country' and feature_value == current_country: # Added country
#                     success_prob += adjustment_value


#     if latency > 150:
#         success_prob -= 0.10
#     if latency > 200:
#         success_prob -= 0.15

#     success_prob = np.clip(success_prob, 0.05, 0.99)

#     if random.random() < success_prob:
#         status = 'success'
#         error_code = None
#     else:
#         status = 'failed'
#         if latency > 200 and random.random() < 0.6:
#             error_code = 'ERR_TIMEOUT'
#         elif random.random() < 0.3:
#             error_code = 'ERR_NETWORK'
#         else:
#             error_code = random.choice([ec for ec in error_codes if ec not in ['ERR_TIMEOUT', 'ERR_NETWORK']])
        
#         latency = max(0, np.random.normal(loc=adjusted_mean_latency * 1.5, scale=std_dev_latency * 2))


#     status_list.append(status)
#     error_code_list.append(error_code)
#     latency_list.append(round(latency, 2))

# data['status'] = status_list
# data['error_code'] = error_code_list
# data['latency_ms'] = latency_list

# df = pd.DataFrame(data)

# df.to_csv("processor_data.csv", index=False)

In [3]:
data= pd.read_csv("processor_data.csv")
data.head(10)

,transaction_id,amount,currency,country,processor,timestamp,payment_method,merchant_category,volume,status,error_code,latency_ms
0,txn_000000,495.37,NGN,Nigeria,Processor_D,2025-07-12 06:11:27.770024,Digital Wallet,Retail,91,success,NaN,115.92
1,txn_000001,59.42,INR,India,Processor_C,2025-06-27 11:24:46.770024,Digital Wallet,Travel,71,success,NaN,118.69
2,txn_000002,685.25,BRL,Brazil,Processor_D,2025-07-23 14:09:41.770024,Debit Card,Digital Goods,105,failed,ERR_NETWORK,184.08
3,txn_000003,798.05,GBP,UK,Processor_A,2025-07-18 10:53:23.770024,Crypto,Digital Goods,95,success,NaN,124.11
4,txn_000004,291.09,EUR,Germany,Processor_B,2025-07-02 13:48:27.770024,Credit Card,Retail,82,success,NaN,95.43
5,txn_000005,350.79,CNY,China,Processor_C,2025-07-12 15:49:45.770024,Debit Card,Travel,78,success,NaN,111.96
6,txn_000006,5.24,EUR,France,Processor_A,2025-07-12 23:08:49.770024,Credit Card,Services,80,success,NaN,106.78
7,txn_000007,793.17,EUR,Germany,Processor_B,2025-07-14 14:51:46.770024,Crypto,Digital Goods,69,success,NaN,81.39
8,txn_000008,253.23,NGN,Nigeria,Processor_A,2025-07-02 15:07:44.770024,Crypto,SaaS,66,success,NaN,94.44
9,txn_000009,441.43,USD,Canada,Processor_D,2025-07-18 14:20:35.770024,Credit Card,Travel,85,failed,ERR_FRAUD,194.05


In [ ]:
data['hour_of_day'] = pd.to_datetime(data['timestamp']).dt.hour

bins = [5, 12, 17, 21, 24]
labels = ['Morning', 'Afternoon', 'Evening', 'Late Night']
data['time_of_day'] = pd.cut(data['hour_of_day'], bins=bins, labels=labels, right=False, include_lowest=True)

In [ ]:
data.head(10)

In [ ]:
data.columns

In [ ]:
data.groupby('processor')['latency_ms'].var().reset_index()

In [ ]:
data['day_of_week'] = pd.to_datetime(data['timestamp']).dt.day_name()
group=data.groupby(['volume', 'day_of_week'])["latency_ms"].mean().reset_index()
group

In [ ]:
data['time_of_day'].value_counts()

In [ ]:
group2=data.groupby('time_of_day')["latency_ms"].mean().reset_index()
group2

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(data=group, x='day_of_week', y='latency_ms',hue='volume')
plt.title('Average Latency by Day of Week and Volume')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(data=group, x='day_of_week', y='latency_ms',hue='volume')
plt.title('Average Latency by Day of Week and Volume')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
current_dir = os.path.dirname(os.path.abspath('model_artifacts'))
model_artifacts = os.path.join(current_dir, 'model_artifacts')

In [ ]:
feature_preprocessor = joblib.load(os.path.join(model_artifacts, 'feature_preprocessor.joblib'))

In [ ]:
df = data.drop(columns=["transaction_id", "error_code", "hour", "hour_of_day", "timestamp", "day_of_week"], axis=1)

In [ ]:
df.columns